In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load data

In [ ]:
# Load data
df_train = pd.read_csv('train.csv', parse_dates=["date"])
df_trans = pd.read_csv('transactions.csv', parse_dates=["date"])
df_stores = pd.read_csv('stores.csv')
df_oil = pd.read_csv('oil.csv', parse_dates=["date"]).rename(columns={"dcoilwtico": "oil"})
df_holi = pd.read_csv('holidays_events.csv', parse_dates=["date"])
df_sub = pd.read_csv('sample_submission.csv')
df_test = pd.read_csv('test.csv', parse_dates=["date"])

# Explore data 

## df_train

### Key takeaways
1. Top 5 product families: Grocery I, Beverages, Produce, Cleaning, Dairy.
2. Promotions and sales have a positive linear relationships.
3. Promotions are most common on Wednesdays, Fridays, and holidays
4. Promotions (total sum) started mid 2015 and have steadily increased since then.

In [ ]:
# Store count = 54
df_train.store_nbr.nunique()

# Same stores in training and test sets
df_train.store_nbr.unique() == df_test.store_nbr.unique()

# Family of products count = 33
df_train.family.nunique()

# Same family of products in training and test sets
df_train.family.unique() == df_test.family.unique()

In [ ]:
# Is every product family reported for every date and store_nbr?
# Store numbers - Every one (54 total) reported for every date in training set
df_store_nbr = df_train.groupby(['date',])['store_nbr'].nunique().reset_index()
df_store_nbr[df_store_nbr['store_nbr'] != 54]

# Product families - Every one (33 total) reported for every date and store number
df_family = df_train.groupby(['date', 'store_nbr'])['family'].nunique().reset_index()
df_family[df_family['family'] != 33]

# How about in the test set?
# Store numbers - Every one (54 total) reported for every date in training set
df_store_nbr2 = df_test.groupby(['date',])['store_nbr'].nunique().reset_index()
df_store_nbr2[df_store_nbr2['store_nbr'] != 54]

# Product families - Every one (33 total) reported for every date and store number
df_family2 = df_test.groupby(['date', 'store_nbr'])['family'].nunique().reset_index()
df_family2[df_family2['family'] != 33]

In [ ]:
# Most popular
# Most popular product families by sales - Top 5: Grocery I, Beverages, Produce, Cleaning, Dairy
df_train.groupby(['family'])['sales'].sum().sort_values(ascending=False)

# Most popular product families by sales by store - Top 5: Beverages, Cleaning, Grocery I, Produce, Dairy
# Little variation by store. Only 13 top 5 entries not in list above.
df_salesFamilyStore = df_train.groupby(['store_nbr', 'family'])['sales'].sum().reset_index()

PopFamily = []
for i in df_salesFamilyStore.store_nbr.unique():
    PopFamily.append(df_salesFamilyStore[df_salesFamilyStore['store_nbr'] == i].nlargest(5, 'sales'))
dfPopFamily = pd.concat(PopFamily)

dfPopFamily.groupby(['family'])['family'].count().sort_values(ascending=False)

In [ ]:
# Most popular promotion items - Top 5: Grocery I, Produce, Beverages, Dairy, Cleaning
df_train.groupby(['family'])['onpromotion'].sum().sort_values(ascending=False)

# Most popular product families by promotion by store - Top 6 (only 6): Beverages, Grocery I, Produce, Cleaning, Dairy, Deli
df_promoFamilyStore = df_train.groupby(['store_nbr', 'family'])['onpromotion'].sum().reset_index()

promoPopFamily = []
for i in df_promoFamilyStore.store_nbr.unique():
    promoPopFamily.append(df_promoFamilyStore[df_promoFamilyStore['store_nbr'] == i].nlargest(5, 'onpromotion'))
dfPromoPopFamily = pd.concat(promoPopFamily)

dfPromoPopFamily.groupby(['family'])['family'].count().sort_values(ascending=False)

In [ ]:
# Is there a relationship between promotions and sales?
# Generally, there is a positive linear relationship between promotions and sales.
# However, product families with few promotions and sales deviate from this relationship.
dfPlotPromoSales = (df_train.groupby(['family'])[['sales', 'onpromotion']]
                    .sum()
                    .sort_values(by='onpromotion'))

x = dfPlotPromoSales['onpromotion']
y = dfPlotPromoSales['sales']

fig = plt.figure(figsize=(5,5))
plt.scatter(x, y)

z = np.polyfit(x, y, 1)
p = np.poly1d(z)
plt.plot(x,p(x),"r--")

plt.xlabel('sales - sum')
plt.ylabel('onpromotion - sum')

subax = fig.add_axes([0.65, 0.2, 0.2, 0.2])
x_labelsize = subax.get_xticklabels()[0].get_size()
y_labelsize = subax.get_yticklabels()[0].get_size()
subax.xaxis.set_tick_params(labelsize=6)
subax.yaxis.set_tick_params(labelsize=6)
subax.scatter(x[:15], y[:15])

plt.show()

In [ ]:
# Are there certain dates popular for promotions?
# Promotions have increased in popularity over time, especially since mid 2015.
dfPlotPromoDates = df_train.groupby(['date'])['onpromotion'].sum().sort_values(ascending=False).reset_index()

fig = plt.figure(figsize=(5,5))
plt.scatter(dfPlotPromoDates['date'], dfPlotPromoDates['onpromotion'])

In [ ]:
# What if I disregard year?
# Looks like promotions spike in the middle and at the end of the year.
# But this might not be true every year since there are many more promotions
# in the later years of the training set.
df_train2 = df_train.copy()

df_train2['month-day'] = df_train2['date'].apply(lambda x: x.replace(year = 2020))

dfPlotPromoMonthDay = df_train2.groupby(['month-day'])['onpromotion'].sum().sort_values(ascending=False).reset_index()

fig = plt.figure(figsize=(5,5))
plt.scatter(dfPlotPromoMonthDay['month-day'], dfPlotPromoMonthDay['onpromotion'])

In [ ]:
# Look at promotions on a yearly basis
# 2013 - no promotions
# 2014-2018 - looks like intraweek variability
# Not too much trend in time of year.
dfPlotPromoYear = dfPlotPromoDates[(dfPlotPromoDates['date'] >= '2017-01-01')
                                   & (dfPlotPromoDates['date'] < '2018-01-01')]
fig = plt.figure(figsize=(5,5))
plt.scatter(dfPlotPromoYear['date'], dfPlotPromoYear['onpromotion'])
plt.show()

In [ ]:
# Look at promotions by day of week
# Wednesdays and Fridays have nearly double the promotions of other days
df_train2['day'] = df_train2['date'].apply(lambda x: x.isoweekday())
dfPlotPromoDay = df_train2.groupby(['day'])['onpromotion'].sum().reset_index()

fig = plt.figure(figsize=(5,5))
plt.scatter(dfPlotPromoDay['day'], dfPlotPromoDay['onpromotion'])
plt.show()

In [ ]:
# How about promotions on public sector paydays (15th and last day of month)
# No trend here. Promotions drop on the 31st since some months don't have 31 days...
# ...I'm looking at the sum of promotions over all dates in the training set
df_train2['day-date'] = df_train2['date'].dt.day

dfPlotPromoDayDate = df_train2.groupby(['day-date'])['onpromotion'].sum().reset_index()

fig = plt.figure(figsize=(5,5))
plt.scatter(dfPlotPromoDayDate['day-date'], dfPlotPromoDayDate['onpromotion'])
plt.show()

In [ ]:
# How about promotions on national holidays
# ~40% more promotions on national and all holidays than other days

# Just national holidays
df_train3 = pd.merge(df_train2, df_holi[df_holi['locale'] == 'National'][['date', 'type']], how='left', on='date')
# All holidays
# df_train3 = pd.merge(df_train2, df_holi[['date', 'type']], how='left', on='date')

df_train3['holiday'] = 1
df_train3.loc[df_train3['type'].isna(), 'holiday'] = 0

dfPlotPromoHoliday = df_train3.groupby(['holiday']).agg({'onpromotion': 'sum', 'date':'nunique'}).reset_index()

dfPlotPromoHoliday['norm'] = dfPlotPromoHoliday['onpromotion'] / dfPlotPromoHoliday['date']

dfPlotPromoHoliday.loc[1, 'norm'] / dfPlotPromoHoliday.loc[0, 'norm']

## df_trans

### Key takeaways
1. Eight of the 54 stores seem to have opened after the start date of the training set (2013-01-01).
2. The capital city (Quito) and state (Pinchincha) are home to 33% and 35% of total stores, respectively.
3. 75% (12 of 16) of states only have stores in one city.
4. 43% (7 of 16) of states only have one store.

In [ ]:
# First date with transactions by store
# Only 8 of 54 stores haven't logged transactions since the beginning of the training set
# Those 8 stores must have been opened after 2013-01-01
dfFirstTrans = df_trans.groupby('store_nbr')['date'].min().reset_index()

dfFirstTrans.groupby('date')['store_nbr'].count()

In [ ]:
# Stores with most transactions
dfTransMax = df_trans.groupby('store_nbr')['transactions'].sum().sort_values(ascending=False).reset_index()

# Store with most transactions has 26x more than store with least transactions
dfTransMax.loc[0, 'transactions'] / dfTransMax.loc[53, 'transactions']

dfTransMax.describe()

In [ ]:
# Transactions by locale
dfTransGeo = pd.merge(dfTransMax, df_stores, how='left', on='store_nbr')

# Group by city
# Stores in Cayambe (1) and Quito (18) average the most transactions per store
dfTransCity = dfTransGeo.groupby(['city']).agg({'transactions': 'sum', 'store_nbr': 'count'}).reset_index()
dfTransCity['norm'] = dfTransCity['transactions'] / dfTransCity['store_nbr']
dfTransCity.sort_values(by='norm', ascending=False)

# Group by state
# Stores in the capital state (Pichincha) average ~25% more transactions than the second most state
dfTransState = dfTransGeo.groupby(['state']).agg({'transactions': 'sum', 'store_nbr': 'count'}).reset_index()
dfTransState['norm'] = dfTransState['transactions'] / dfTransState['store_nbr']
dfTransState.sort_values(by='norm', ascending=False)

# 1/3 of stores are in Quito
dfTransCity[dfTransCity['city'] == 'Quito']['store_nbr'] / dfTransCity.store_nbr.sum()

# 35% are in the state of the capital city, Quito (Pichincha)
dfTransState[dfTransState['state'] == 'Pichincha']['store_nbr'] / dfTransState.store_nbr.sum()

In [ ]:
# Group by type
# There are only 5 store types. Type A averages more than double the number of transactions
# per store than second place Type D
dfTransType = dfTransGeo.groupby(['type']).agg({'transactions': 'sum', 'store_nbr': 'count'}).reset_index()
dfTransType['norm'] = dfTransType['transactions'] / dfTransType['store_nbr']
dfTransType.sort_values(by='norm', ascending=False)

# Group by cluster
# There are 17 clusters with 1-7 stores each. The average number of transactions
# per store per cluster varies wildly (2.8 +/- 1.6).
dfTransClus = dfTransGeo.groupby(['cluster']).agg({'transactions': 'sum', 'store_nbr': 'count'}).reset_index()
dfTransClus['norm'] = dfTransClus['transactions'] / dfTransClus['store_nbr']
dfTransClus.sort_values(by='norm', ascending=False)

dfTransClus.describe()

# States and cities can have stores of different types
dfTransGeo.groupby(['state', 'city', 'type']).agg({'transactions': 'sum', 'store_nbr': 'count'}).reset_index()

In [ ]:
# 75% of states only have stores in one city
dfCityState = dfTransGeo.groupby(['state',]).agg({'store_nbr': 'nunique', 'city': 'nunique'}).reset_index()
dfCityState[dfCityState['city'] == 1]['state'].count() / dfCityState.state.count()

# ~43% of states only have one store
dfCityState[dfCityState['store_nbr'] == 1]['state'].count() / dfCityState.state.count()

## df_oil

### Key takeaways
1. Oil prices moved from an average of 99 (dollars? per unit) to 47 (dollars? per unit) suddenly at the end of 2014.
2. Prices were relatively stable otherwise with the exception of 3 or 4 swings.
3. 43 dates are missing a daily oil price.
4. Weekends are not included in dataset.

In [ ]:
# Plot oil prices
x = df_oil['date']
y = df_oil['oil']

fig = plt.figure(figsize=(5,5))
plt.scatter(x, y)

# Average 'high' price
x1 = x[:450]
y1 = [y[:450].mean()]*450
plt.plot(x1, y1,"r--")
plt.text(16350, 98, round(y1[0], 2), color='red')

# Average 'low' price
x2 = x[500:]
y2 = [y[500:].mean()]*718
plt.plot(x2, y2,"r--")
plt.text(16200, 46.5, round(y2[0], 2), color='red')

plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Daily Oil Price')

In [ ]:
# Check for dates missing a daily price
# 43 dates missing a daily price
# Weekends are not included in dataset
df_oil[df_oil['oil'].isna()].date.count()

# Weekdays 5 and 6 (Saturday and Sunday) are missing from the dataset
df_oil.date.dt.weekday.reset_index().groupby(['date'])['date'].count()

## df_holi

### Key definitions
1.  A holiday on a row with column "transferred" equal to True was celebrated on a different date. <br>
    This date was treated like a regular day. The corresponding row with column "type" equal to Transfer <br>
    contains the date the holiday was celebrated.
2. "type" equal to Bridge is an additional day off added to the holiday.
3. "type" equal to Work Day is a day worked that is not normally worked (e.g., Saturday) <br>
    to make up for  a Bridge day.

### Key takeaways:
1. Most holidays are at the national or local level.
2. Holidays/events by year are not stable. 2012 is light. 2014 has many events related to the World Cup <br>
    2016 has many events related to the earthquake.
3. Most holidays are in the summer months or at the end of the year.
4. Holiday counts dip on Tuesdays and Wednesdays are are relatively stable on other days of the week.

In [ ]:
# Breakdown of holidays by type
df_holi.groupby(['type'])['date'].count()

# Exclude multiple holidays on same date
df_holi.groupby(['type'])['date'].nunique()

In [ ]:
# Count of transferred holidays: 12
len(df_holi[df_holi['transferred'] == True])

# Were all holidays marked as "transferred" = True celebrated elsewhere?
# Yes
df_holi[df_holi['transferred'] == True]
df_holi[df_holi['type'] == 'Transfer']

len(df_holi[df_holi['transferred'] == True]) == len(df_holi[df_holi['type'] == 'Transfer'])

In [ ]:
# Count of "type" equal to Bridge and Work Day both equal to 5.
# Are all five of each coordinated pairs? 
# Yes
df_holi[df_holi['type'] == 'Bridge']
df_holi[df_holi['type'] == 'Work Day']

In [ ]:
# Breakdown of holidays by locale
# Includes 12 duplicates due to transfers
df_holi.groupby(['locale'])['date'].count()

In [ ]:
# Is the count of holidays/events per year relatively stable?
# No. Extra events in 2014 due to the World Cup and in 2016 due to the earthquake.
# 2012 is a light year.
df_holiYear = df_holi.copy()
df_holiYear['year'] = df_holiYear.date.dt.year
df_holiYear.groupby(['year']).date.count()

In [ ]:
# Holidays/events by month
# Most holidays in summer months or at end of year
df_holiYear['month'] = df_holiYear.date.dt.month
df_holiYear.groupby(['month']).date.count()

In [ ]:
# Holidays/events by day of week
# Count dips slightly on Tuesdays and Wednesdays
df_holiYear['day'] = df_holiYear['date'].apply(lambda x: x.isoweekday())
df_holiYear.groupby(['day']).date.count()

# Prepare data

In [ ]:
# First training data date = 2013-01-01
train_start = df_train.date.min()
train_start

# Last training data date = 2017-08-15
train_end = df_train.date.max()
train_end

# Check for missing dates (discontinuities) in training data -> Christmas Day
missing_dates = pd.date_range(train_start, train_end).difference(df_train.date.unique())
missing_dates = missing_dates.strftime("%Y-%m-%d").tolist()
print('Dates missing from training dataset: ', missing_dates)